# [6.4 Planning as a CSP](http://artint.info/2e/html/ArtInt2e.Ch6.S4.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=92) (page 92)

## About

In forward planning, the search is constrained by the initial state and only uses the goal as a stopping criterion and as a source for heuristics. In regression planning, the search is constrained by the goal and only uses the start state as a stopping criterion and as a source for heuristics. By converting the problem to a constraint satisfaction problem (CSP), the initial state can be used to prune what is not reachable and the goal to prune what is not useful. The CSP will be defined for a finite number of steps; the number of steps (also called __horizon__) can be adjusted to find the shortest plan. We can apply any method we learnt before (see [Chapter 4](https://artint.info/2e/html/ArtInt2e.Ch4.html)) in order to solve this CSP and thus find a plan.

You can run each cell by selecting it and pressing *Ctrl+Enter* in Windows or *Shift+Return* in Mac. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. For more information, check out our AISpace2 [Tutorial](https://aispace2.github.io/AISpace2/tutorial.html).

Feel free to modify our codes either in this notebook or somewhere outside (e.g. python files in `/aipython/`). If you want to modify our codes outside, you might find [this](https://aispace2.github.io/AISpace2/tutorial.html#tutorial-faq-why-update-aipython-not-reflect) helpful for how your changes can take effect.

You need to run the following command to import our pre-defined problems. You can also define your own problems ([how?](https://aispace2.github.io/AISpace2/tutorial.html#tutorial-planning-construct-yourself)). 

In [ ]:
# Run this first to import dependencies
from aipython.stripsProblem import strips_simple1, strips_simple2, strips_simple3, strips_blocks1, strips_blocks2, strips_blocks3

You can click on an inconsistent arc to make it consistent, or click buttons such as "Step" to randomly select an arc to make consistent. After all arcs are consistent, you can click on a variable with more than one value in its domain to split it.

In [ ]:
from aipython.cspConsistency import Con_solver
from aipython.stripsCSPPlanner import CSP_from_STRIPS

cspFromSTRIPS = CSP_from_STRIPS(planning_problem=strips_simple2, horizon=2)
solver = Con_solver(csp=cspFromSTRIPS)

# Visualization options
solver.sleep_time = 0.2 # The time, in seconds, between each step in auto solving
solver.line_width = 4.0 # The thickness of edges
solver.text_size = 15 # The fontsize of the text
solver.detail_level = 1 # 0=no text, 1=truncated text, 2=full text

# Display the widget
display(solver)
solver.solve_one()